In [ ]:
using Einsum
using Random
using LinearAlgebra

In [ ]:
function ×₃(T::Array{Float64, 3}, v::Vector{Float64})
    @einsum M[i,j] := T[i,j,k]*v[k] #contract the 3rd index of a tensor with a vector
    return M
end

function mu_gnmf(X, T; maxiter=800, tol=5e-4, λ=0, ϵ=1e-8) #multipicative updated nonnegative matrix factorization
    #using LinearAlgebra
    
    # Initilization
    m, n = size(X)
    r, N, p = size(T)
    @assert n==N "Missmatch between the second dimention of X and T"
    A = abs.(randn((m, r)))
    b = abs.(randn((p,)))
    i = 0

    # Updates
    while (norm(X - A*(T×₃b))/norm(X) > tol) && (i < maxiter)
        i += 1
        @show i
        #@einsum b[q] = b[q] * T[u,v,q]*A[w,u]*X[w,v] / (T[i,j,q]*A[l,i]*A[l,s]*T[s,j,t]*b[t] + ϵ + λ*b[q]) #(W'*V ) ./ (W'*W*H  .+ ϵ + λ.*H) #update b
        #b_old = b
        for q ∈ 1:p
            TT = T[:,:,q]
            b[q] *= tr(TT'*A'*X) / (tr(TT'*A'*A*(T×₃b)) + ϵ + λ*b[q]) #can replace b with b_old?
        end
        B = (T×₃b)
        A .*= (X *B') ./ (A *B*B' .+ ϵ + λ.*A) #update A
    end

    return (A, b, i)
end
#Random.seed!(314)
m,n,r,p = (100,100,10,5)
A = abs.(randn((m, r)))
b = abs.(randn((p,)))
T = abs.(randn((r,n,p)))
T[T.>=1] .= 1
T[T.< 1] .= 0
X = A*(T×₃b)
(AA, bb, i) = mu_gnmf(X, T)